<a href="https://colab.research.google.com/github/Ishakasar/Recommender-engine/blob/main/Isha_Kasar_Team_2_Capstone_Project_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# <b><u> Project Title : Build a recommender engine that reviews customer ratings and purchase history to recommend items and improve sales. </u></b>

### Amazon.com is one of the largest electronic commerce and cloud computing companies.

### Just a few Amazon related facts:

### They lost $4.8 million in August 2013, when their website went down for 40 mins. They hold the patent on 1-Click buying, and licenses it to Apple. Their Phoenix fulfilment centre is a massive 1.2 million square feet. Amazon relies heavily on a Recommendation engine that reviews customer ratings and purchase history to recommend items and improve sales.


### This is a dataset related to over 2 Million customer reviews and ratings of Beauty related products sold on their website.

### It contains

* ### the unique UserId (Customer Identification),
* ### the product ASIN (Amazon's unique product identification code for each product),
* ### Ratings (ranging from 1-5 based on customer satisfaction) and
* ### the Timestamp of the rating (in UNIX time)

### This dataset contains product reviews and metadata from Amazon, including 142.8 million reviews spanning May 1996 - July 2014.

### This dataset includes reviews (ratings, text, helpfulness votes), product metadata (descriptions, category information, price, brand, and image features), and links (also viewed/also bought graphs).


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import numpy as np
import scipy
import pandas as pd
import math
import random
import sklearn
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from scipy.sparse.linalg import svds
import matplotlib.pyplot as plt

In [3]:
df1=pd.read_csv('/content/drive/MyDrive/AlmaBetter/Cohort Aravali/Module 4/Week 4/ratings_Beauty.csv')

In [4]:
df2=pd.read_json('/content/drive/MyDrive/AlmaBetter/Cohort Aravali/Module 4/Week 4/reviews_Beauty_5.json.gz', lines = True)

In [5]:
df1.head()

,UserId,ProductId,Rating,Timestamp
0,A39HTATAQ9V7YF,0205616461,5.0,1369699200
1,A3JM6GV9MNOF9X,0558925278,3.0,1355443200
2,A1Z513UWSAAO0F,0558925278,5.0,1404691200
3,A1WMRR494NWEWV,0733001998,4.0,1382572800
4,A3IAAVS479H7M7,0737104473,1.0,1274227200


In [6]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2023070 entries, 0 to 2023069
Data columns (total 4 columns):
 #   Column     Dtype  
---  ------     -----  
 0   UserId     object 
 1   ProductId  object 
 2   Rating     float64
 3   Timestamp  int64  
dtypes: float64(1), int64(1), object(2)
memory usage: 61.7+ MB


In [7]:
df1.describe()

,Rating,Timestamp
count,2.023070e+06,2.023070e+06
mean,4.149036e+00,1.360389e+09
std,1.311505e+00,4.611860e+07
min,1.000000e+00,9.087552e+08
25%,4.000000e+00,1.350259e+09
50%,5.000000e+00,1.372810e+09
75%,5.000000e+00,1.391472e+09
max,5.000000e+00,1.406074e+09


In [8]:
df1.isnull().sum()

UserId       0
ProductId    0
Rating       0
Timestamp    0
dtype: int64

In [9]:
df2.head()

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime
0,A1YJEY40YUW4SE,7806397051,Andrea,"[3, 4]",Very oily and creamy. Not at all what I expect...,1,Don't waste your money,1391040000,"01 30, 2014"
1,A60XNB876KYML,7806397051,Jessica H.,"[1, 1]",This palette was a decent price and I was look...,3,OK Palette!,1397779200,"04 18, 2014"
2,A3G6XNM240RMWA,7806397051,Karen,"[0, 1]",The texture of this concealer pallet is fantas...,4,great quality,1378425600,"09 6, 2013"
3,A1PQFP6SAJ6D80,7806397051,Norah,"[2, 2]",I really can't tell what exactly this thing is...,2,Do not work on my face,1386460800,"12 8, 2013"
4,A38FVHZTNQ271F,7806397051,Nova Amor,"[0, 0]","It was a little smaller than I expected, but t...",3,It's okay.,1382140800,"10 19, 2013"


In [10]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 198502 entries, 0 to 198501
Data columns (total 9 columns):
 #   Column          Non-Null Count   Dtype 
---  ------          --------------   ----- 
 0   reviewerID      198502 non-null  object
 1   asin            198502 non-null  object
 2   reviewerName    197116 non-null  object
 3   helpful         198502 non-null  object
 4   reviewText      198502 non-null  object
 5   overall         198502 non-null  int64 
 6   summary         198502 non-null  object
 7   unixReviewTime  198502 non-null  int64 
 8   reviewTime      198502 non-null  object
dtypes: int64(2), object(7)
memory usage: 13.6+ MB


In [11]:
df2.describe()

,overall,unixReviewTime
count,198502.000000,1.985020e+05
mean,4.190391,1.364450e+09
std,1.166580,3.841937e+07
min,1.000000,1.023840e+09
25%,4.000000,1.354493e+09
50%,5.000000,1.372205e+09
75%,5.000000,1.391213e+09
max,5.000000,1.406074e+09


In [12]:
df2.isnull().sum()

reviewerID           0
asin                 0
reviewerName      1386
helpful              0
reviewText           0
overall              0
summary              0
unixReviewTime       0
reviewTime           0
dtype: int64

In [13]:
df1.head()

,UserId,ProductId,Rating,Timestamp
0,A39HTATAQ9V7YF,0205616461,5.0,1369699200
1,A3JM6GV9MNOF9X,0558925278,3.0,1355443200
2,A1Z513UWSAAO0F,0558925278,5.0,1404691200
3,A1WMRR494NWEWV,0733001998,4.0,1382572800
4,A3IAAVS479H7M7,0737104473,1.0,1274227200


In [14]:
df2.head()

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime
0,A1YJEY40YUW4SE,7806397051,Andrea,"[3, 4]",Very oily and creamy. Not at all what I expect...,1,Don't waste your money,1391040000,"01 30, 2014"
1,A60XNB876KYML,7806397051,Jessica H.,"[1, 1]",This palette was a decent price and I was look...,3,OK Palette!,1397779200,"04 18, 2014"
2,A3G6XNM240RMWA,7806397051,Karen,"[0, 1]",The texture of this concealer pallet is fantas...,4,great quality,1378425600,"09 6, 2013"
3,A1PQFP6SAJ6D80,7806397051,Norah,"[2, 2]",I really can't tell what exactly this thing is...,2,Do not work on my face,1386460800,"12 8, 2013"
4,A38FVHZTNQ271F,7806397051,Nova Amor,"[0, 0]","It was a little smaller than I expected, but t...",3,It's okay.,1382140800,"10 19, 2013"


In [15]:
df2[['reviewerID','asin']].duplicated().value_counts()

False    198502
dtype: int64

In [16]:
df=df1.sample(frac = 0.4)

In [17]:
users_interactions_count_df = df.groupby(['UserId', 'ProductId']).size().groupby('UserId').size()
print('# of users: %d' % len(users_interactions_count_df))

interaction_df = users_interactions_count_df[users_interactions_count_df >= 5].reset_index()[['UserId']]
print('# of users with at least 5 interactions: %d' % len(interaction_df))

# of users: 594436
# of users with at least 5 interactions: 11335


In [18]:
users_interactions_count_df.head()

UserId
A00008821J0F472NDY6A2    1
A000186437REL8X2RW8UW    1
A00029263J863WSR0TDRS    1
A00031961JI1CBNV98TW     1
A00088021N0VU5Z2UM73L    1
dtype: int64

In [19]:
interaction_df.head()

,UserId
0,A0078719IR14X3NNUG0F
1,A02155413BVL8D0G7X6DN
2,A029527620Q3SK5XW16RR
3,A03364251DGXSGA9PSR99
4,A042274212BJJVOBS4Q85


In [20]:
''

''

In [21]:
interactions_from_selected_users_df = df1.merge(interaction_df, 
               how = 'right',
               left_on = 'UserId',
               right_on = 'UserId')
print('# of interactions from users with at least 5 interactions: %d' % len(interactions_from_selected_users_df))

# of interactions from users with at least 5 interactions: 196962


In [22]:
interactions_from_selected_users_df.head(20)

,UserId,ProductId,Rating,Timestamp
0,A0078719IR14X3NNUG0F,B001CE6T76,4.0,1367798400
1,A0078719IR14X3NNUG0F,B005Y6F4WO,5.0,1367280000
2,A0078719IR14X3NNUG0F,B0069FDR96,5.0,1367193600
3,A0078719IR14X3NNUG0F,B006L1DNWY,5.0,1366416000
4,A0078719IR14X3NNUG0F,B007HORL1I,5.0,1370044800
5,A0078719IR14X3NNUG0F,B0083QNBCM,5.0,1367798400
6,A0078719IR14X3NNUG0F,B008RRX9CE,5.0,1366416000
7,A0078719IR14X3NNUG0F,B008SPOKIW,5.0,1367452800
8,A0078719IR14X3NNUG0F,B0098T2JWG,5.0,1368835200
9,A0078719IR14X3NNUG0F,B0098T3D5I,5.0,1368835200


In [23]:
df1.shape

(2023070, 4)

In [24]:
interactions_from_selected_users_df.shape

(196962, 4)

In [25]:
import math
def smooth_user_preference(x):
    return math.log(1+x, 2)
    
interactions_full_df = interactions_from_selected_users_df.groupby(['UserId', 'ProductId'])['Rating'].sum().apply(smooth_user_preference).reset_index()
print('# of unique user/item interactions: %d' % len(interactions_full_df))
interactions_full_df.head()

# of unique user/item interactions: 196962


,UserId,ProductId,Rating
0,A0078719IR14X3NNUG0F,B001CE6T76,2.321928
1,A0078719IR14X3NNUG0F,B005Y6F4WO,2.584963
2,A0078719IR14X3NNUG0F,B0069FDR96,2.584963
3,A0078719IR14X3NNUG0F,B006L1DNWY,2.584963
4,A0078719IR14X3NNUG0F,B007HORL1I,2.584963


In [26]:
interactions_train_df, interactions_test_df = train_test_split(interactions_full_df,
                                   stratify=interactions_full_df['UserId'], 
                                   test_size=0.30,
                                   random_state=42)

In [27]:

print('# interactions on Train set: %d' % len(interactions_train_df))
print('# interactions on Test set: %d' % len(interactions_test_df))

# interactions on Train set: 137873
# interactions on Test set: 59089


##Creating pivot table

In [28]:
#Creating a sparse pivot table with users in rows and items in columns
pivot_matrix_df = interactions_train_df.pivot(index='UserId', 
                                                          columns='ProductId', 
                                                          values='Rating').fillna(0)

In [29]:
pivot_matrix_df.shape

(11335, 55249)

In [30]:
df2.shape

(198502, 9)

In [31]:
pivot_matrix_df.head()

ProductId,0762451459,1304139212,130414643X,1304174867,1304351475,1304482596,1304482685,1304495396,1304511073,1304511081,1304511111,1304511138,1304511146,1304511154,1304622223,1304622452,1304622460,1304622657,1304622665,1304651029,1304651118,1304665704,1304665720,130466578X,4057368825,5297000963,5357954771,6022600790,6162751473,7806397051,8901110814,9510213861,9511181564,957696718X,9746427962,9747842297,9749377397,9759091062,9788071139,978807118X,...,B00KVE7R0Q,B00KVO2JWC,B00KWE08Q0,B00KWL83Q0,B00KWR2IV0,B00KXENYY2,B00KXF9WPG,B00KXG8JF4,B00KY9ODMS,B00KYCKPBI,B00KYWDP1A,B00L0C529Q,B00L0GJOMS,B00L0RT0XA,B00L0YCBIY,B00L1CALVO,B00L252RBM,B00L2JKUY4,B00L2K53BS,B00L2KVF9W,B00L3GD6H4,B00L3K91OW,B00L3KPPRE,B00L3LB0IG,B00L3LBUGI,B00L3V2904,B00L4I5PJS,B00L5BXG36,B00L5JHZJO,B00L5KTZ0K,B00L6Q3BH6,B00L7BX4FE,B00L8OYGUC,B00LAI1FYG,B00LFWG7EU,B00LG63DOM,B00LJEACWC,B00LLPT4HI,B00LMOT7X4,B00LNOKBYW
UserId,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
A0078719IR14X3NNUG0F,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A02155413BVL8D0G7X6DN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A029527620Q3SK5XW16RR,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A03364251DGXSGA9PSR99,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A042274212BJJVOBS4Q85,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [32]:
pivot_matrix = pivot_matrix_df.values
pivot_matrix[:10]

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [33]:
users_ids = list(pivot_matrix_df.index)
users_ids[:10]

['A0078719IR14X3NNUG0F',
 'A02155413BVL8D0G7X6DN',
 'A029527620Q3SK5XW16RR',
 'A03364251DGXSGA9PSR99',
 'A042274212BJJVOBS4Q85',
 'A07739331MLU43RRRK661',
 'A08409921GVN7C8RD97CV',
 'A0908131Z7BWYSMRQ16T',
 'A100UD67AHFODS',
 'A100WO06OQR8BQ']

##SVD

In [34]:
# The number of factors to factor the user-item matrix.
NUMBER_OF_FACTORS_MF = 30

#Performs matrix factorization of the original user item matrix
U, sigma, Vt = svds(pivot_matrix, k = NUMBER_OF_FACTORS_MF)

In [35]:
pivot_matrix.shape

(11335, 55249)

In [36]:
U.shape

(11335, 10)

In [37]:
sigma = np.diag(sigma)
sigma.shape

(10, 10)

In [38]:
Vt.shape

(10, 55249)

In [39]:
all_user_predicted_ratings = np.dot(np.dot(U, sigma), Vt) 
all_user_predicted_ratings

array([[ 2.90201109e-05,  4.23574062e-07,  4.23574062e-07, ...,
         9.40795223e-04,  1.13814536e-03,  3.64050379e-04],
       [ 1.03858807e-03,  1.12397169e-07,  1.12397169e-07, ...,
        -3.44822415e-03,  1.01506893e-03,  2.34498847e-05],
       [ 2.02706881e-04,  4.52434113e-08,  4.52434113e-08, ...,
         6.42053798e-04,  2.82472767e-04,  2.69098639e-05],
       ...,
       [ 8.69747395e-07,  2.55792088e-08,  2.55792088e-08, ...,
         1.58888384e-04,  6.24218893e-05,  2.15291218e-05],
       [ 2.49559819e-05,  6.69410771e-09,  6.69410771e-09, ...,
        -5.93541019e-05,  5.13797389e-05,  5.47334487e-06],
       [ 5.57345212e-04,  1.62161693e-07,  1.62161693e-07, ...,
         1.34083240e-03,  8.59535723e-04,  1.18399393e-04]])

In [40]:
all_user_predicted_ratings.shape

(11335, 55249)

In [41]:
#Converting the reconstructed matrix back to a Pandas dataframe
preds_df = pd.DataFrame(all_user_predicted_ratings, columns = pivot_matrix_df.columns, index=users_ids).transpose()
preds_df.head()

,A0078719IR14X3NNUG0F,A02155413BVL8D0G7X6DN,A029527620Q3SK5XW16RR,A03364251DGXSGA9PSR99,A042274212BJJVOBS4Q85,A07739331MLU43RRRK661,A08409921GVN7C8RD97CV,A0908131Z7BWYSMRQ16T,A100UD67AHFODS,A100WO06OQR8BQ,A100ZQDV7L8PVV,A1016Z89IM29SK,A101NJYKW882Y0,A10220YORA6MUU,A1022QCIW8Z720,A102JNFLL0KW7I,A103979529MRJY0U56QI4,A103BJIOJSDJL1,A103WXT3CHVY0H,A103ZVTCQUF348,A1043CGOMHVZQ7,A1047EDJ84IMAS,A104FF3C0UPVA5,A1051DBTLWP5A2,A1057ZABSPSAXE,A105GPLAKST4B4,A105R38CAPDBGU,A105S56ODHGJEK,A106E1N0ZQ4D9W,A106Q1A6LGU4DP,A106YK8JBW1JBP,A107PAB5U6L76G,A1084V41BP7RDH,A108B8OUQ0GHNF,A108LWN98Q5V41,A108S8GV9IJJ96,A108VB7LEU7M6W,A108W62SUWQCL4,A108XP24UESKSV,A1093UZ147FN0Q,...,AZFJDE00JTMPT,AZGK12P6XWY27,AZIDXJ9F7A83Z,AZIFU5UI0G71E,AZIS8K2M1HYPJ,AZJ8PVOKAJS5,AZJFLPEG7D09G,AZK41OIXZ425Y,AZL43EZU06FKF,AZL4Z6X1436BM,AZLE2XFH3JUNG,AZLG1RXKT7EZ8,AZM1SDKXBVJYR,AZM5T3BRXN8ZA,AZMMFZKJB8PV6,AZMY6E8B52L2T,AZN28ZLXYIJDE,AZNNIVT2R0W0L,AZNS7TH82KH9K,AZOK7E2RKF9XD,AZOO27I3Z0IQO,AZP87257LO2A7,AZPHLOJLKZDBD,AZPHYNPEZDMIO,AZR10WSWFD4B5,AZR9KW2ZA9MMA,AZRL9I4SG5ADA,AZTZ7SIIRXLXE,AZUEKJ6P9TFSJ,AZUH2MX87LX7J,AZUI6YY673GW5,AZV2AG96CRJ26,AZW9UD7Z7J0WL,AZWJAXOQMB8EK,AZX1JTTIUYZX4,AZYA6NBTF2843,AZZ91VNEU3XYX,AZZIOU48NE8PC,AZZNK89PXD006,AZZT1ERHBSNQ8
ProductId,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0762451459,2.902011e-05,1.038588e-03,2.027069e-04,1.571658e-04,2.353661e-05,1.627182e-05,7.149241e-07,8.195819e-05,1.123008e-05,2.008569e-03,8.336852e-04,3.516254e-06,1.391626e-05,8.652543e-05,7.731153e-06,-7.427847e-05,0.000248,2.024835e-04,-8.505171e-05,2.696113e-05,9.315704e-06,3.532753e-04,1.151588e-04,6.853843e-05,8.169694e-05,9.242942e-07,3.160329e-05,2.519985e-03,4.389002e-05,5.569638e-04,3.450195e-05,9.303703e-05,1.527605e-04,5.759361e-05,5.927926e-06,1.236365e-04,1.915773e-04,5.479279e-05,1.369513e-03,2.336341e-05,...,2.189428e-05,3.756676e-05,3.332695e-04,2.334987e-04,2.055286e-04,3.634494e-04,-5.779552e-05,7.741777e-05,1.643768e-04,4.935896e-04,3.356175e-05,2.127680e-04,2.470352e-05,1.631502e-05,-9.162391e-05,3.875897e-04,4.516367e-04,7.961121e-05,2.113120e-06,1.211467e-06,6.441203e-04,3.842466e-05,-9.185850e-05,1.308397e-06,4.246473e-05,9.170006e-06,8.427969e-05,1.009960e-04,-8.142615e-05,5.584754e-06,3.767044e-05,6.348480e-05,-4.375692e-05,7.149590e-05,3.243000e-05,8.214481e-05,4.180946e-05,8.697474e-07,2.495598e-05,5.573452e-04
1304139212,4.235741e-07,1.123972e-07,4.524341e-08,8.508639e-08,9.816004e-09,7.606653e-09,2.840281e-09,2.823086e-07,2.072863e-09,-1.931126e-07,5.644831e-08,1.173041e-09,6.850299e-09,3.185990e-08,2.275991e-07,-2.281746e-08,0.000002,4.438380e-08,-2.383452e-09,6.801307e-09,2.147838e-07,2.834069e-07,3.975857e-08,2.165074e-08,3.408243e-08,1.821490e-08,6.474681e-08,5.075158e-07,2.752923e-08,-1.719338e-08,1.402605e-08,2.786225e-08,5.139518e-08,8.278465e-09,5.643697e-10,7.483804e-07,4.386051e-08,1.687309e-08,2.513085e-07,1.392635e-08,...,8.491385e-09,1.603727e-08,1.843013e-07,7.135090e-08,2.695294e-08,1.272084e-07,3.734897e-07,2.293823e-07,1.382812e-08,1.244819e-07,8.621796e-07,4.548579e-08,7.018082e-09,1.134040e-08,-4.583474e-08,1.615379e-07,1.342961e-07,3.118413e-08,2.073167e-09,1.281306e-09,1.723794e-07,9.525710e-09,3.791151e-07,6.614402e-07,1.749255e-08,2.358799e-09,2.040459e-07,2.107524e-08,-2.601936e-08,2.450262e-08,8.074439e-09,2.638995e-08,-1.341854e-08,3.483579e-08,1.169896e-08,3.515870e-08,1.939903e-08,2.557921e-08,6.694108e-09,1.621617e-07
130414643X,4.235741e-07,1.123972e-07,4.524341e-08,8.508639e-08,9.816004e-09,7.606653e-09,2.840281e-09,2.823086e-07,2.072863e-09,-1.931126e-07,5.644831e-08,1.173041e-09,6.850299e-09,3.185990e-08,2.275991e-07,-2.281746e-08,0.000002,4.438380e-08,-2.383452e-09,6.801307e-09,2.147838e-07,2.834069e-07,3.975857e-08,2.165074e-08,3.408243e-08,1.821490e-08,6.474681e-08,5.075158e-07,2.752923e-08,-1.719338e-08,1.402605e-08,2.786225e-08,5.139518e-08,8.278465e-09,5.643697e-10,7.483804e-07,4.386051e-08,1.687309e-08,2.513085e-07,1.392635e-08,...,8.491385e-09,1.603727e-08,1.84301

In [42]:
df2['reviewerID'].value_counts()

A2V5R832QCSOMX    204
ALNFHVS3SC4FV     192
AKMEY1BSHSDG7     182
A3KEZLJ59C1JVH    154
ALQGOMOY1F5X9     150
                 ... 
A2ZT5T3FBCOXQV      5
A1NCBAR3ANANGR      5
A2RE87ZZU4D5JK      5
A1WA5SRH3ZYIQJ      5
AZM2QKIIGG9I2       5
Name: reviewerID, Length: 22363, dtype: int64